<a href="https://colab.research.google.com/github/kojikokojiko/portfoilio/blob/master/M5/val%E8%A8%93%E7%B7%B4WI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# light gbmのGPU版を生成するコード　使わないでい
# !git clone --recursive https://github.com/Microsoft/LightGBM
# %cd /content/LightGBM/
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package
# !sudo python setup.py install --precompile

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:


!ls

drive  sample_data


In [0]:

%cd /content/drive/My Drive/kaggle/M5/iwase

/content/drive/.shortcut-targets-by-id/18e73-MFsSq-bfNImjjh-axhWYXPQE-7c/kaggle/M5/iwase


In [0]:
import  lightgbm as lgb
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

In [0]:



# weights = pd.read_pickle('./c_weight.pickle')
# weight1 = weights[weights.index ==0]
# weight2 = weights[weights.index ==1]
# weight1=weight1.values
# weight2=weight2.values

In [0]:


# NUM_ITEMS = 30490
# DAYS_PRED = 28
# def wrmsse(preds, data):
    
#     # this function is calculate for last 28 days to consider the non-zero demand period
    
#     # actual obserbed values / 正解ラベル
#     y_true = data.get_label()
    
#     y_true = y_true[-(NUM_ITEMS * DAYS_PRED):]
#     preds = preds[-(NUM_ITEMS * DAYS_PRED):]
#     # number of columns
#     num_col = DAYS_PRED
    
#     # reshape data to original array((NUM_ITEMS*num_col,1)->(NUM_ITEMS, num_col) ) / 推論の結果が 1 次元の配列になっているので直す
#     reshaped_preds = preds.reshape(num_col, NUM_ITEMS).T
#     reshaped_true = y_true.reshape(num_col, NUM_ITEMS).T
    
          
#     train = weight_mat_csr*np.c_[reshaped_preds, reshaped_true]
    
#     score = np.sum(
#                 np.sqrt(
#                     np.mean(
#                         np.square(
#                             train[:,:num_col] - train[:,num_col:])
#                         ,axis=1) / weight1) * weight2)
    
#     return 'wrmsse', score, False

In [0]:
def seed_everything(seed=0):
    # random.seed(seed)
    np.random.seed(seed)

In [0]:
#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913              # End day of our train set
P_HORIZON   = 28                 # Prediction horizon

In [0]:
remove_features = [
                  #  デフォルト
                   'id',
                   'state_id',
                   'store_id',
                   'date',
                   'wm_yr_wk',
                   'd',
                   TARGET,

                  #  随時追加
                  'sales_lag_tmp_1',
                  'sales_lag_tmp_7',
                   
                   




                   

                   ]

In [0]:
# ここで、学習するストアを選択
# 学習しないものはCtr/でメモに

STORE_IDS=[
#  'CA_1',
#  'CA_2',
#  'CA_3',
#  'CA_4',
#  'TX_1',
#  'TX_2',
#  'TX_3',
 'WI_1',
 'WI_2',
 'WI_3'
]

# ここ変えるの忘れない！！！！

In [0]:
# ハイパラ、特徴量をいじったらVERを更新
VER=4

In [0]:
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    # 'metric': 'custom',
                    'metric':'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': 1,

                } 

In [0]:
#################
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible

In [0]:
# for i in range(1,29):
for store in STORE_IDS:
  print(store)
  grid_df=pd.merge(pd.read_pickle('../iwase_output/val/val_'+store+'.pkl'),
                   pd.read_pickle('../iwase_output/val/val_lag_'+store+'.pkl'),
                   how='left',
                   on=['d','id','sales']
                   )
    
  print('loaded')
   # dを数字にする式
  grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)
   # 訓練、検証、テストのマスクを作成
  train_mask = grid_df['d']<=END_TRAIN
  valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
  preds_mask = grid_df['d']>(END_TRAIN-100)

  features_columns = [col for col in list(grid_df) if col not in remove_features]

  train_data = lgb.Dataset(grid_df[train_mask][features_columns], label=grid_df[train_mask][TARGET])
  valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], label=grid_df[valid_mask][TARGET])
  
  grid_df = grid_df[preds_mask].reset_index(drop=True)
  keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
  grid_df = grid_df[keep_cols]
  grid_df.to_pickle('../testpkl/val/ver'+str(VER)+'/val_test_v'+str(VER)+store+'.pkl')
  print('to_pickle　完了')

  del grid_df
  gc.collect()
  seed_everything(SEED)

  print('train_start')
  estimator = lgb.train(lgb_params,
                        train_data,
                        valid_sets = [train_data,valid_data],
                        verbose_eval = 100,
                        early_stopping_rounds = 50,
                        # feval= wrmsse,
                        )
    
  print('train_finished')
  model_name = '../model/val/ver'+str(VER)+'/val_lgb_model_v'+str(VER)+'_'+store+'.bin' #変更点
  pickle.dump(estimator, open(model_name, 'wb'))

  del train_data, valid_data, estimator
  gc.collect()

WI_1
loaded
to_pickle　完了
train_start


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 1.71251	valid_1's rmse: 1.59521
[200]	training's rmse: 1.67102	valid_1's rmse: 1.57801
[300]	training's rmse: 1.65526	valid_1's rmse: 1.57034
[400]	training's rmse: 1.6445	valid_1's rmse: 1.56453
[500]	training's rmse: 1.63585	valid_1's rmse: 1.55967
[600]	training's rmse: 1.62816	valid_1's rmse: 1.55517
[700]	training's rmse: 1.62144	valid_1's rmse: 1.55149
[800]	training's rmse: 1.61508	valid_1's rmse: 1.54744
[900]	training's rmse: 1.60909	valid_1's rmse: 1.5441
[1000]	training's rmse: 1.60406	valid_1's rmse: 1.54042
[1100]	training's rmse: 1.59897	valid_1's rmse: 1.5373
[1200]	training's rmse: 1.59428	valid_1's rmse: 1.53426
[1300]	training's rmse: 1.58979	valid_1's rmse: 1.5309
[1400]	training's rmse: 1.58515	valid_1's rmse: 1.52801
Did not meet early stopping. Best iteration is:
[1400]	training's rmse: 1.58515	valid_1's rmse: 1.52801
train_finished
WI_2
loaded
to_pickle　完了
train_start


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 2.67788	valid_1's rmse: 2.70272
[200]	training's rmse: 2.61435	valid_1's rmse: 2.59834
[300]	training's rmse: 2.58627	valid_1's rmse: 2.56391
[400]	training's rmse: 2.56687	valid_1's rmse: 2.54146
[500]	training's rmse: 2.54994	valid_1's rmse: 2.52449
[600]	training's rmse: 2.53536	valid_1's rmse: 2.50716
[700]	training's rmse: 2.52181	valid_1's rmse: 2.49551
[800]	training's rmse: 2.50914	valid_1's rmse: 2.48313
[900]	training's rmse: 2.49805	valid_1's rmse: 2.47226
[1000]	training's rmse: 2.48694	valid_1's rmse: 2.45991
[1100]	training's rmse: 2.47673	valid_1's rmse: 2.44814
[1200]	training's rmse: 2.46711	valid_1's rmse: 2.43719
[1300]	training's rmse: 2.45707	valid_1's rmse: 2.42922
[1400]	training's rmse: 2.44749	valid_1's rmse: 2.42148
Did not meet early stopping. Best iteration is:
[1400]	training's rmse: 2.44749	valid_1's rmse: 2.42148
train_finished
WI_3
loaded
to_pickle　完了
train_start


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 2.43619	valid_1's rmse: 1.9232
[200]	training's rmse: 2.32382	valid_1's rmse: 1.84951
[300]	training's rmse: 2.28052	valid_1's rmse: 1.82617
[400]	training's rmse: 2.25292	valid_1's rmse: 1.81222
[500]	training's rmse: 2.23081	valid_1's rmse: 1.80505
[600]	training's rmse: 2.21268	valid_1's rmse: 1.7986
[700]	training's rmse: 2.19776	valid_1's rmse: 1.7922
[800]	training's rmse: 2.18345	valid_1's rmse: 1.7865
[900]	training's rmse: 2.17139	valid_1's rmse: 1.78275
[1000]	training's rmse: 2.16013	valid_1's rmse: 1.77756
[1100]	training's rmse: 2.15057	valid_1's rmse: 1.77402
[1200]	training's rmse: 2.14101	valid_1's rmse: 1.77009
[1300]	training's rmse: 2.13177	valid_1's rmse: 1.76461
[1400]	training's rmse: 2.12257	valid_1's rmse: 1.76044
Did not meet early stopping. Best iteration is:
[1400]	training's rmse: 2.12257	valid_1's rmse: 1.76044
train_finished
